In [1]:
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers


Using TensorFlow backend.


In [2]:
sys.setrecursionlimit(40000)

#  <font colot = 'black'> The following functions(except for the last two cells are modified from github:https://github.com/you359/Keras-FasterRCNN. And model are modified to work on the specific dataset.</font>

In [3]:
def format_img_size(img, C):
    """ formats the image size based on config """
    img_min_side = float(C.im_size)
    (height,width,_) = img.shape

    if width <= height:
        ratio = img_min_side/width
        new_height = int(ratio * height)
        new_width = int(img_min_side)
    else:
        ratio = img_min_side/height
        new_width = int(ratio * width)
        new_height = int(img_min_side)
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    return img, ratio

In [4]:
def format_img_channels(img, C):
    """ formats the image channels based on config """
    img = img[:, :, (2, 1, 0)]
    img = img.astype(np.float32)
    img[:, :, 0] -= C.img_channel_mean[0]
    img[:, :, 1] -= C.img_channel_mean[1]
    img[:, :, 2] -= C.img_channel_mean[2]
    img /= C.img_scaling_factor
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img


In [5]:
def format_img(img, C):
    """ formats an image for model prediction based on config """
    img, ratio = format_img_size(img, C)
    img = format_img_channels(img, C)
    return img, ratio

In [6]:
def get_real_coordinates(ratio, x1, y1, x2, y2):

    real_x1 = int(round(x1 // ratio))
    real_y1 = int(round(y1 // ratio))
    real_x2 = int(round(x2 // ratio))
    real_y2 = int(round(y2 // ratio))

    return (real_x1, real_y1, real_x2 ,real_y2)

#  <font colot = 'black'> Change C.model_path in above cell to change the model weights. './model_frcnn.hdf5' is the simpler one and './model_frcnn_2.hdf5' (default) is the better one
</font>

In [7]:
test_path = '../re_image/'
ano_path = '../annotate.txt'
num_rois = 32
config_filename = "config.pickle"
network = 'vgg'

config_output_filename = config_filename
with open(config_output_filename, 'rb') as f_in:
    C = pickle.load(f_in)
if C.network == 'resnet50':
    import keras_frcnn.resnet as nn
elif C.network == 'vgg':
    import keras_frcnn.vgg as nn
C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False

img_path = test_path

class_mapping = C.class_mapping

if 'bg' not in class_mapping:
    class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}
C.num_rois = int(num_rois)

if C.network == 'resnet50':
    num_features = 1024
elif C.network == 'vgg':
    num_features = 512

if K.image_dim_ordering() == 'th':
    input_shape_img = (3, None, None)
    input_shape_features = (num_features, None, None)
else:
    input_shape_img = (None, None, 3)
    input_shape_features = (None, None, num_features)


img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=input_shape_features)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
model_classifier_only = Model([feature_map_input, roi_input], classifier)

model_classifier = Model([feature_map_input, roi_input], classifier)

#C.model_path = './model_frcnn.hdf5'
print('Loading weights from {}'.format(C.model_path))
model_rpn.load_weights(C.model_path, by_name=True)
model_classifier.load_weights(C.model_path, by_name=True)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

{0: 'Face', 1: 'bg'}
Loading weights from ./model_frcnn.hdf5


#  <font colot = 'black'> Uncomment the last line of the above to save images to required path.</font>

In [8]:
detection_mat = {}

all_imgs = []

classes = {}

bbox_threshold = 0.8

visualise = True

for idx, img_name in enumerate(sorted(os.listdir(img_path))):
    if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
        continue
    print(img_name)
    st = time.time()
    filepath = os.path.join(img_path,img_name)

    img = cv2.imread(filepath)

    X, ratio = format_img(img, C)

    if K.image_dim_ordering() == 'tf':
        X = np.transpose(X, (0, 2, 3, 1))

    # get the feature maps and output from the RPN
    [Y1, Y2, F] = model_rpn.predict(X)


    R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)

    # convert from (x1,y1,x2,y2) to (x,y,w,h)
    R[:, 2] -= R[:, 0]
    R[:, 3] -= R[:, 1]

    # apply the spatial pyramid pooling to the proposed regions
    bboxes = {}
    probs = {}

    for jk in range(R.shape[0]//C.num_rois + 1):
        ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
        if ROIs.shape[1] == 0:
            break

        if jk == R.shape[0]//C.num_rois:
            #pad R
            curr_shape = ROIs.shape
            target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
            ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
            ROIs_padded[:, :curr_shape[1], :] = ROIs
            ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
            ROIs = ROIs_padded

        [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

        for ii in range(P_cls.shape[1]):

            if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                continue

            cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

            if cls_name not in bboxes:
                bboxes[cls_name] = []
                probs[cls_name] = []

            (x, y, w, h) = ROIs[0, ii, :]

            cls_num = np.argmax(P_cls[0, ii, :])
            try:
                (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
                tx /= C.classifier_regr_std[0]
                ty /= C.classifier_regr_std[1]
                tw /= C.classifier_regr_std[2]
                th /= C.classifier_regr_std[3]
                x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
            except:
                pass
            bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
            probs[cls_name].append(np.max(P_cls[0, ii, :]))

    all_dets = []

    for key in bboxes:
        bbox = np.array(bboxes[key])

        new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)

        
        for jk in range(new_boxes.shape[0]):
            (x1, y1, x2, y2) = new_boxes[jk,:]
            if new_probs[jk] < 0.95:
                continue
            (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)

            #cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)

            textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
            all_dets.append((key,100*new_probs[jk]))

            (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
            textOrg = (real_x1, real_y1-0)

            #cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
            #cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
            #cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)

    croped_img = img[real_y1:real_y2,real_x1:real_x2]
    detection_mat['../re_image/'+img_name] = [real_y1,real_x1,real_y2,real_x2]
    print('Elapsed time = {}'.format(time.time() - st))
    print(all_dets)
    #cv2.imwrite('../detection_2/'+img_name,croped_img)

001_.jpg
Elapsed time = 1.9310698509216309
[('Face', 99.98691082000732)]
001_d.jpg
Elapsed time = 0.2495577335357666
[('Face', 99.99669790267944)]
001_dt.jpg
Elapsed time = 0.2303297519683838
[('Face', 99.99203681945801)]
001_t.jpg
Elapsed time = 0.22423934936523438
[('Face', 99.88282322883606)]
002_.jpg
Elapsed time = 0.22649669647216797
[('Face', 99.98804330825806)]
002_d.jpg
Elapsed time = 0.22225356101989746
[('Face', 99.99321699142456)]
002_dt.jpg
Elapsed time = 0.24140596389770508
[('Face', 99.99313354492188)]
002_t.jpg
Elapsed time = 0.2299947738647461
[('Face', 99.97166991233826)]
003_.jpg
Elapsed time = 0.22530531883239746
[('Face', 99.97288584709167)]
003_d.jpg
Elapsed time = 0.235795259475708
[('Face', 99.99712705612183)]
003_dt.jpg
Elapsed time = 0.23790478706359863
[('Face', 99.98883008956909)]
003_t.jpg
Elapsed time = 0.22209525108337402
[('Face', 99.92280006408691)]
004_.jpg
Elapsed time = 0.24190831184387207
[('Face', 99.9915361404419)]
004_d.jpg
Elapsed time = 0.238818

Elapsed time = 0.23180246353149414
[('Face', 99.9894380569458)]
028_t.jpg
Elapsed time = 0.2278752326965332
[('Face', 99.86293315887451)]
029_.jpg
Elapsed time = 0.22546148300170898
[('Face', 99.84468817710876)]
029_d.jpg
Elapsed time = 0.2302868366241455
[('Face', 99.98999834060669)]
029_dt.jpg
Elapsed time = 0.23245525360107422
[('Face', 99.9776303768158)]
029_t.jpg
Elapsed time = 0.2214796543121338
[('Face', 99.89702701568604)]
030_.jpg
Elapsed time = 0.23346161842346191
[('Face', 99.96616840362549)]
030_d.jpg
Elapsed time = 0.22233247756958008
[('Face', 99.95492100715637)]
030_dt.jpg
Elapsed time = 0.24578285217285156
[('Face', 99.88634586334229)]
030_t.jpg
Elapsed time = 0.22845029830932617
[]
031_.jpg
Elapsed time = 0.23233509063720703
[('Face', 99.99160766601562)]
031_d.jpg
Elapsed time = 0.22145342826843262
[('Face', 99.99265670776367)]
031_dt.jpg
Elapsed time = 0.2442340850830078
[('Face', 99.99305009841919)]
031_t.jpg
Elapsed time = 0.23251748085021973
[('Face', 99.9716579914

Elapsed time = 0.22484731674194336
[('Face', 99.95675683021545)]
056_dt.jpg
Elapsed time = 0.2170097827911377
[('Face', 99.9434769153595)]
056_t.jpg
Elapsed time = 0.23220396041870117
[('Face', 99.8609185218811)]
057_.jpg
Elapsed time = 0.23144245147705078
[('Face', 99.91344809532166)]
057_d.jpg
Elapsed time = 0.22072863578796387
[('Face', 99.97336268424988)]
057_dt.jpg
Elapsed time = 0.22736811637878418
[('Face', 99.98499155044556)]
057_t.jpg
Elapsed time = 0.22173619270324707
[('Face', 99.05732274055481)]
058_.jpg
Elapsed time = 0.23610734939575195
[('Face', 99.78001713752747)]
058_d.jpg
Elapsed time = 0.22949957847595215
[('Face', 99.99785423278809)]
058_dt.jpg
Elapsed time = 0.2230837345123291
[('Face', 99.99381303787231)]
058_t.jpg
Elapsed time = 0.22818279266357422
[('Face', 99.99309778213501)]
059_.jpg
Elapsed time = 0.22536087036132812
[('Face', 99.97525811195374)]
059_d.jpg
Elapsed time = 0.24414634704589844
[('Face', 99.94984865188599)]
059_dt.jpg
Elapsed time = 0.23128271102

Elapsed time = 0.24725866317749023
[('Face', 99.9798595905304)]
084_.jpg
Elapsed time = 0.24901247024536133
[('Face', 99.98635053634644)]
084_d.jpg
Elapsed time = 0.2256784439086914
[('Face', 99.99421834945679)]
084_dt.jpg
Elapsed time = 0.22422266006469727
[('Face', 99.99901056289673)]
084_t.jpg
Elapsed time = 0.22299408912658691
[('Face', 99.98550415039062)]
085_.jpg
Elapsed time = 0.23616647720336914
[('Face', 99.95390176773071)]
085_d.jpg
Elapsed time = 0.22446537017822266
[('Face', 99.99337196350098)]
085_dt.jpg
Elapsed time = 0.23332571983337402
[('Face', 99.99455213546753)]
085_t.jpg
Elapsed time = 0.2241382598876953
[('Face', 99.9901533126831)]
086_.jpg
Elapsed time = 0.23881816864013672
[('Face', 99.93120431900024)]
086_d.jpg
Elapsed time = 0.23807215690612793
[('Face', 99.97214674949646)]
086_dt.jpg
Elapsed time = 0.22380733489990234
[('Face', 99.98354911804199)]
086_t.jpg
Elapsed time = 0.2626824378967285
[('Face', 99.96836185455322)]
087_.jpg
Elapsed time = 0.23352527618408

Elapsed time = 0.2264411449432373
[('Face', 99.99910593032837)]
111_dt.jpg
Elapsed time = 0.24356722831726074
[('Face', 99.99926090240479)]
111_t.jpg
Elapsed time = 0.23917436599731445
[('Face', 99.99310970306396)]
112_.jpg
Elapsed time = 0.24145293235778809
[('Face', 99.85931515693665)]
112_d.jpg
Elapsed time = 0.22801804542541504
[('Face', 99.98822212219238)]
112_dt.jpg
Elapsed time = 0.2523794174194336
[('Face', 99.9745786190033)]
112_t.jpg
Elapsed time = 0.23877596855163574
[('Face', 99.81512427330017)]
113_.jpg
Elapsed time = 0.24338245391845703
[('Face', 99.48201179504395)]
113_d.jpg
Elapsed time = 0.23029661178588867
[('Face', 99.9921441078186)]
113_dt.jpg
Elapsed time = 0.2299051284790039
[('Face', 99.98040795326233)]
113_t.jpg
Elapsed time = 0.23803353309631348
[('Face', 99.90480542182922)]
114_.jpg
Elapsed time = 0.2385420799255371
[('Face', 99.67369437217712)]
114_d.jpg
Elapsed time = 0.2436070442199707
[('Face', 99.99698400497437)]
114_dt.jpg
Elapsed time = 0.23252964019775

Elapsed time = 0.23944878578186035
[('Face', 99.7703492641449)]
139_d.jpg
Elapsed time = 0.22241783142089844
[('Face', 99.989914894104)]
139_dt.jpg
Elapsed time = 0.2237412929534912
[('Face', 99.99467134475708)]
139_t.jpg
Elapsed time = 0.22854995727539062
[('Face', 99.96941089630127)]
140_.jpg
Elapsed time = 0.24410295486450195
[('Face', 99.60110187530518)]
140_d.jpg
Elapsed time = 0.24579405784606934
[('Face', 99.98955726623535)]
140_dt.jpg
Elapsed time = 0.2395610809326172
[('Face', 99.99133348464966)]
140_t.jpg
Elapsed time = 0.25028061866760254
[('Face', 97.99740314483643)]
141_.jpg
Elapsed time = 0.2799344062805176
[('Face', 99.98880624771118)]
141_d.jpg
Elapsed time = 0.2676386833190918
[('Face', 99.9951958656311)]
141_dt.jpg
Elapsed time = 0.24910235404968262
[('Face', 99.98311996459961)]
141_t.jpg
Elapsed time = 0.2386188507080078
[('Face', 99.98419284820557)]
142_.jpg
Elapsed time = 0.24267578125
[('Face', 99.92353916168213)]
142_d.jpg
Elapsed time = 0.24385762214660645
[('Fa

Elapsed time = 0.25687718391418457
[('Face', 99.84671473503113)]
167_.jpg
Elapsed time = 0.24229931831359863
[('Face', 99.92536902427673)]
167_d.jpg
Elapsed time = 0.2248368263244629
[('Face', 99.98929500579834)]
167_dt.jpg
Elapsed time = 0.23476886749267578
[('Face', 99.97733235359192)]
167_t.jpg
Elapsed time = 0.24567174911499023
[('Face', 99.71169233322144)]
168_.jpg
Elapsed time = 0.23340988159179688
[('Face', 99.62410926818848)]
168_d.jpg
Elapsed time = 0.23818159103393555
[('Face', 99.98561143875122)]
168_dt.jpg
Elapsed time = 0.22484898567199707
[('Face', 99.99233484268188)]
168_t.jpg
Elapsed time = 0.23487639427185059
[('Face', 99.25479292869568)]
169_.jpg
Elapsed time = 0.25229644775390625
[('Face', 99.62579011917114)]
169_d.jpg
Elapsed time = 0.23083209991455078
[('Face', 99.97969269752502)]
169_dt.jpg
Elapsed time = 0.2427806854248047
[('Face', 99.98703002929688)]
169_t.jpg
Elapsed time = 0.2348179817199707
[('Face', 99.93661046028137)]
170_.jpg
Elapsed time = 0.270685195922

Elapsed time = 0.25449681282043457
[('Face', 99.99860525131226)]
194_dt.jpg
Elapsed time = 0.25398683547973633
[('Face', 99.99504089355469)]
194_t.jpg
Elapsed time = 0.2469315528869629
[('Face', 99.94433522224426)]
195_.jpg
Elapsed time = 0.25688767433166504
[('Face', 99.93163347244263)]
195_d.jpg
Elapsed time = 0.22582507133483887
[('Face', 99.99855756759644)]
195_dt.jpg
Elapsed time = 0.22335600852966309
[('Face', 99.98996257781982)]
195_t.jpg
Elapsed time = 0.253040075302124
[('Face', 99.94240999221802)]


In [9]:
# From https://blog.csdn.net/leviopku/article/details/81629492
def compute_iou(rec1, rec2):
    """
    computing IoU
    :param rec1: (y0, x0, y1, x1), which reflects
            (top, left, bottom, right)
    :param rec2: (y0, x0, y1, x1)
    :return: scala value of IoU
    """
    # computing area of each rectangles
    S_rec1 = (rec1[2] - rec1[0]) * (rec1[3] - rec1[1])
    S_rec2 = (rec2[2] - rec2[0]) * (rec2[3] - rec2[1])
 
    # computing the sum_area
    sum_area = S_rec1 + S_rec2
 
    # find the each edge of intersect rectangle
    left_line = max(rec1[1], rec2[1])
    right_line = min(rec1[3], rec2[3])
    top_line = max(rec1[0], rec2[0])
    bottom_line = min(rec1[2], rec2[2])
 
    # judge if there is an intersect
    if left_line >= right_line or top_line >= bottom_line:
        return 0
    else:
        intersect = (right_line - left_line) * (bottom_line - top_line)
        return intersect / (sum_area - intersect)

# <font color = 'black'>calculate mean IOU</font>

In [11]:
f = open('../annotate.txt')
line = f.readline().strip().split(',')

#print(detection_mat[line[0]])
total_IOU = 0
cnt = 0
while line != ['']:
    x =[0,0,0,0]
    if len(line) < 4:     
        if line == ['']:
            break
        line = f.readline().strip().split(',')
        continue
    x[0] = int(line[2])
    x[1] = int(line[1])
    x[2] = int(line[4])
    x[3] = int(line[3])
    total_IOU+=(compute_iou(x,detection_mat[line[0]]))
    cnt += 1
    line = f.readline().strip().split(',')
f.close()
print('mean_IOU',total_IOU/cnt)

mean_IOU 0.8718056317391457


model_2 mean IOU: 90.47